In [18]:
import glob
import os
import pandas as pd

In [19]:
scorep_result_dir = './scorep-results'

# get all files in the directory
scorep_result_dir = './scorep-results'
scorep_result_dir = glob.glob(os.path.join(scorep_result_dir, '*'))

In [20]:
otf2_trace = "./scorep-results/bt.C.4.mpi_io_full" + '/traces.otf2'

In [21]:
import otf2.reader
from otf2.events import Enter
def extract_unique_functions(trace_file_path: str) -> set:
    """
    Extract unique function names from an OTF2 trace file.
    
    Args:
        trace_file_path (str): Path to the OTF2 trace file
            Example: './path/to/traces.otf2'
    
    Returns:
        set[str]: Set of unique function names found in the trace
            Example: {'MPI_Init', 'MPI_Finalize', 'MPI_Barrier'}
    
    Raises:
        ValueError: If file path doesn't end with .otf2
        FileNotFoundError: If trace file doesn't exist
    """
    # Validate input file
    if not trace_file_path.endswith('.otf2'):
        raise ValueError("Input file must be an .otf2 trace file")
    
    if not os.path.exists(trace_file_path):
        raise FileNotFoundError(f"Trace file not found: {trace_file_path}")

    unique_functions = set()
    
    try:
        with otf2.reader.open(trace_file_path) as trace:
            for _, event in trace.events:
                if isinstance(event, Enter):
                    unique_functions.add(event.region.name)
    except Exception as e:
        raise RuntimeError(f"Error reading trace file: {str(e)}")
            
    return unique_functions

In [22]:
extract_unique_functions(otf2_trace)

{'MPI_Allreduce',
 'MPI_Barrier',
 'MPI_Bcast',
 'MPI_Comm_dup',
 'MPI_Comm_rank',
 'MPI_Comm_size',
 'MPI_File_close',
 'MPI_File_delete',
 'MPI_File_open',
 'MPI_File_read_at_all',
 'MPI_File_set_view',
 'MPI_File_write_at_all',
 'MPI_Finalize',
 'MPI_Init',
 'MPI_Irecv',
 'MPI_Isend',
 'MPI_Reduce',
 'MPI_Wait',
 'MPI_Waitall'}

In [23]:
import pandas as pd
from collections import defaultdict
from otf2.events import Enter, Leave

def calculate_accumulated_function_time(otf2_trace: str) -> pd.DataFrame:
    """
    Calculate timing metrics from an OTF2 trace file.
    
    Returns DataFrame with columns:
    - Function: Name of the function
    - Total Time (s): Total time spent in function
    - Call Count: Number of invocations
    - Average Time (s): Mean time per call
    
    Usage:
    >>> otf2_trace = "./scorep-results/bt.C.4.mpi_io_full/traces.otf2"
    >>> df = calculate_accumulated_function_time(otf2_trace)
    >>> print(df)
    
    Results:
    >>> df.head()
          Function  Total Time (s)  Call Count  Average Time (s)
    0  MPI_Init          0.123456          10          0.012346
    1  MPI_Finalize      0.234567          20          0.011728
    2  MPI_Barrier       0.345678          30          0.011523
    3  MPI_Bcast         0.456789          40          0.011420
    4  MPI_Reduce        0.567890          50          0.011358
    """
    # Input validation
    if not otf2_trace.endswith('.otf2'):
        raise ValueError("Input file must be an .otf2 trace file")
    
    # Initialize tracking dictionaries
    metrics = {
        'times': defaultdict(float),  # Track total time spent in each function
        'counts': defaultdict(int)    # Track number of calls to each function
    }
    call_stacks = defaultdict(list)
    
    # Process trace events
    with otf2.reader.open(otf2_trace) as trace:
        resolution = trace.timer_resolution
        
        for location, event in trace.events:
            if isinstance(event, Enter):
                call_stacks[location].append((event.region, event.time))
                metrics['counts'][event.region] += 1
                
            elif isinstance(event, Leave):
                if not call_stacks[location]:
                    raise RuntimeError(f"Unmatched Leave event in {location.name}")
                    
                region, start_time = call_stacks[location].pop()
                if region != event.region:
                    raise RuntimeError(f"Mismatched Enter/Leave in {location.name}")
                
                duration = (event.time - start_time) / resolution
                metrics['times'][region] += duration
    

    function_names = [fn.name for fn in metrics['times']]
    total_times = list(metrics['times'].values())
    call_counts = [metrics['counts'][fn] for fn in metrics['times']]
    avg_times = [metrics['times'][fn] / metrics['counts'][fn] for fn in metrics['times']]          
    return pd.DataFrame({
        'Function': function_names,
        'Total Time (s)': total_times,
        'Call Count': call_counts,
        'Average Time (s)': avg_times
    })


In [24]:
calculate_accumulated_function_time(otf2_trace)

,Function,Total Time (s),Call Count,Average Time (s)
0,MPI_Init,3.571143,4,8.927858e-01
1,MPI_Comm_size,0.000007,4,1.679048e-06
2,MPI_Comm_rank,0.000002,4,4.985714e-07
3,MPI_Comm_dup,0.002375,8,2.969209e-04
4,MPI_Bcast,0.003023,32,9.445419e-05
5,MPI_File_delete,0.035998,1,3.599807e-02
6,MPI_Barrier,0.112276,12,9.356348e-03
7,MPI_File_open,0.233338,8,2.916728e-02
8,MPI_File_set_view,0.002926,8,3.657445e-04
9,MPI_Irecv,0.013122,9672,1.356701e-06


In [25]:
def beautify_df(df: pd.DataFrame, top_n: int = 10) -> pd.DataFrame:
    """
    Beautify the DataFrame by sorting and aggregating smaller values.
    
    Parameters:
    It then aggregates the rows beyond the top_n into a single row labeled 'others'.
    The resulting DataFrame is reset to have a continuous index.
    
    Parameters:
    dataframe (pd.DataFrame): The input DataFrame with at least a 'Total Time (s)' column.
    top_n (int): The number of top rows to retain before aggregating the rest. Default is 10.
    
    Returns:
    pd.DataFrame: A beautified DataFrame with the top_n rows and an aggregated 'others' row.
    
    Usage:
    >>> df = pd.DataFrame({
    >>>     'Function': ['A', 'B', 'C', 'D', 'E'],
    >>>     'Total Time (s)': [10, 9, 8, 7, 6],
    >>>     'Call Count': [100, 90, 80, 70, 60],
    >>>     'Average Time (s)': [0.1, 0.1, 0.1, 0.1, 0.1]
    >>> })
    >>> beautified_df = beautify_df(df, top_n=3)
    >>> print(beautified_df)
    
    Results:
    >>> beautified_df
      Function  Total Time (s)  Call Count  Average Time (s)
    0        A           10.00         100              0.10
    1        B            9.00          90              0.10
    2        C            8.00          80              0.10
    3   others           22.00         270              0.10
    """
    # Sort the DataFrame by 'Total Time (s)' in descending order
    sorted_df = df.sort_values(['Total Time (s)'], ascending=[False])
    
    # Aggregate rows beyond the top_n into a single row labeled 'others'
    remaining_df = sorted_df[top_n:].drop(columns='Function').agg('sum').to_frame().T
    remaining_df.insert(0, 'Function', 'others')
    
    # Concatenate top rows with the aggregated 'others' row
    result_df = pd.concat([sorted_df.head(top_n), remaining_df]).reset_index(drop=True)

    return result_df.round(4)

In [26]:
beautify_df(calculate_accumulated_function_time(otf2_trace), top_n=5)

,Function,Total Time (s),Call Count,Average Time (s)
0,MPI_File_write_at_all,20.8652,160.0,0.1304
1,MPI_File_read_at_all,12.7506,160.0,0.0797
2,MPI_Wait,10.1933,9648.0,0.0011
3,MPI_Init,3.5711,4.0,0.8928
4,MPI_Isend,0.6676,9672.0,0.0001
5,others,1.1138,10741.0,0.0769


In [27]:
def list_all_events(trace_file_path: str):
    """
    List all events from an OTF2 trace file.
    
    Args:
        trace_file_path (str): Path to the OTF2 trace file
    
    Returns:
        list: A list of all events found in the trace file
    """
    # Validate input file
    if not trace_file_path.endswith('.otf2'):
        raise ValueError("Input file must be an .otf2 trace file")
    
    if not os.path.exists(trace_file_path):
        raise FileNotFoundError(f"Trace file not found: {trace_file_path}")
    
    event_list = []
    with otf2.reader.open(trace_file_path) as trace:
        for _, event in trace.events:
            event_list.append(event)
    
    return event_list

# Example usage
all_events = list_all_events(otf2_trace)
print(len(all_events))


100611


In [28]:
def calculate_io_bandwidth(trace_file_path: str):
    """
    Calculate I/O bandwidth from OTF2 trace file.
    
    Args:
        trace_file_path (str): Path to the OTF2 trace file
        
    Returns:
        dict: Dictionary containing bandwidth metrics
    """
    # Initialize storage for I/O operations
    io_ops = {
        'read': [],
        'write': []
    }
    
    with otf2.reader.open(trace_file_path) as trace:
        # Track ongoing operations by matching_id
        ongoing_ops = {}
        
        for _, event in trace.events:
            if isinstance(event, otf2.events.IoOperationBegin):
                # Store operation start info
                ongoing_ops[event.matching_id] = {
                    'start_time': event.time,
                    'bytes_requested': event.bytes_request,
                    'mode': event.mode
                }
                
            elif isinstance(event, otf2.events.IoOperationComplete):
                if event.matching_id in ongoing_ops:
                    start_info = ongoing_ops[event.matching_id]
                    duration = event.time - start_info['start_time']
                    bytes_transferred = event.bytes_result
                    
                    # Calculate bandwidth in MB/s
                    if duration > 0:
                        bandwidth = (bytes_transferred / (1024 * 1024)) / (duration * 1e-9)
                        
                        # Store operation details
                        op_type = 'read' if start_info['mode'] == otf2.IoOperationMode.READ else 'write'
                        io_ops[op_type].append({
                            'bytes': bytes_transferred,
                            'duration_ns': duration,
                            'bandwidth_mbs': bandwidth
                        })
                    
                    del ongoing_ops[event.matching_id]

    # Calculate aggregate metrics
    metrics = {
        'read': {
            'total_bytes': sum(op['bytes'] for op in io_ops['read']),
            'total_duration_ns': sum(op['duration_ns'] for op in io_ops['read']),
            'avg_bandwidth_mbs': 0,
            'peak_bandwidth_mbs': max((op['bandwidth_mbs'] for op in io_ops['read']), default=0)
        },
        'write': {
            'total_bytes': sum(op['bytes'] for op in io_ops['write']),
            'total_duration_ns': sum(op['duration_ns'] for op in io_ops['write']), 
            'avg_bandwidth_mbs': 0,
            'peak_bandwidth_mbs': max((op['bandwidth_mbs'] for op in io_ops['write']), default=0)
        }
    }
    
    # Calculate average bandwidth
    for op_type in ['read', 'write']:
        if metrics[op_type]['total_duration_ns'] > 0:
            metrics[op_type]['avg_bandwidth_mbs'] = \
                (metrics[op_type]['total_bytes'] / (1024 * 1024)) / \
                (metrics[op_type]['total_duration_ns'] * 1e-9)
    
    return metrics

In [29]:
def calculate_io_bandwidth_by_size(trace_file_path: str):
    """
    Calculate I/O bandwidth grouped by transaction sizes from OTF2 trace file.
    
    Args:
        trace_file_path (str): Path to the OTF2 trace file
        
    Returns:
        dict: Dictionary containing bandwidth metrics per transaction size
    """
    # Initialize storage for I/O operations by size
    io_ops_by_size = {
        'read': {},  # Will store operations grouped by size
        'write': {}  # Will store operations grouped by size
    }
    
    with otf2.reader.open(trace_file_path) as trace:
        # Track ongoing operations by matching_id
        ongoing_ops = {}
        
        for _, event in trace.events:
            if isinstance(event, otf2.events.IoOperationBegin):
                # Store operation start info
                ongoing_ops[event.matching_id] = {
                    'start_time': event.time,
                    'bytes_requested': event.bytes_request,
                    'mode': event.mode
                }
                
            elif isinstance(event, otf2.events.IoOperationComplete):
                if event.matching_id in ongoing_ops:
                    start_info = ongoing_ops[event.matching_id]
                    duration = event.time - start_info['start_time']
                    bytes_transferred = event.bytes_result
                    
                    # Calculate bandwidth in MB/s
                    if duration > 0:
                        bandwidth = (bytes_transferred / (1024 * 1024)) / (duration * 1e-9)
                        
                        # Store operation details grouped by size
                        op_type = 'read' if start_info['mode'] == otf2.IoOperationMode.READ else 'write'
                        size_key = bytes_transferred  # Use actual transfer size as key
                        
                        if size_key not in io_ops_by_size[op_type]:
                            io_ops_by_size[op_type][size_key] = []
                            
                        io_ops_by_size[op_type][size_key].append({
                            'bytes': bytes_transferred,
                            'duration_ns': duration,
                            'bandwidth_mbs': bandwidth
                        })
                    
                    del ongoing_ops[event.matching_id]

    # Calculate metrics per size
    metrics_by_size = {
        'read': {},
        'write': {}
    }
    
    for op_type in ['read', 'write']:
        for size, operations in io_ops_by_size[op_type].items():
            metrics_by_size[op_type][size] = {
                'count': len(operations),
                'total_bytes': sum(op['bytes'] for op in operations),
                'total_duration_ns': sum(op['duration_ns'] for op in operations),
                'avg_bandwidth_mbs': 0,
                'min_bandwidth_mbs': min((op['bandwidth_mbs'] for op in operations), default=0),
                'max_bandwidth_mbs': max((op['bandwidth_mbs'] for op in operations), default=0),
                'std_bandwidth_mbs': 0
            }
            
            # Calculate average bandwidth
            if metrics_by_size[op_type][size]['total_duration_ns'] > 0:
                metrics_by_size[op_type][size]['avg_bandwidth_mbs'] = \
                    (metrics_by_size[op_type][size]['total_bytes'] / (1024 * 1024)) / \
                    (metrics_by_size[op_type][size]['total_duration_ns'] * 1e-9)
            
            # Calculate standard deviation of bandwidth
            if len(operations) > 1:
                mean = metrics_by_size[op_type][size]['avg_bandwidth_mbs']
                variance = sum((op['bandwidth_mbs'] - mean) ** 2 for op in operations) / (len(operations) - 1)
                metrics_by_size[op_type][size]['std_bandwidth_mbs'] = variance ** 0.5
    
    return metrics_by_size

In [30]:
calculate_io_bandwidth_by_size(otf2_trace)

{'read': {42515280: {'count': 40,
   'total_bytes': 1700611200,
   'total_duration_ns': 6687167402,
   'avg_bandwidth_mbs': 242.5285814061953,
   'min_bandwidth_mbs': 75.58547166298375,
   'max_bandwidth_mbs': 298.51505473358645,
   'std_bandwidth_mbs': 34.97532679883168}},
 'write': {42515280: {'count': 40,
   'total_bytes': 1700611200,
   'total_duration_ns': 10879508135,
   'avg_bandwidth_mbs': 149.07192526611522,
   'min_bandwidth_mbs': 64.62763295368534,
   'max_bandwidth_mbs': 162.2347998607631,
   'std_bandwidth_mbs': 15.680638135255899}}}

In [31]:
otf2.definitions

<module 'otf2.definitions' from '/usr/local/lib/python3.12/site-packages/otf2/definitions.py'>

In [32]:
from enum import Enum

class Otf2Paradigm(Enum):
    """
    Enumeration for different I/O paradigms with their identifications in OTF2 trace files.
    
    Attributes:
        MPIIO (str): Represents the MPI-IO paradigm.
        POSIX (str): Represents the POSIX paradigm.
    """
    MPIIO = 'MPI-IO'
    POSIX = 'POSIX'

In [33]:
def calculate_mpi_io_bandwidth_by_size(trace_file_path: str, paradigm: Otf2Paradigm) -> dict:
    """
    Calculate I/O bandwidth specifically for MPI I/O operations grouped by transaction size.
    
    Args:
        trace_file_path (str): Path to the OTF2 trace file
        
    Returns:
        dict: Dictionary containing bandwidth metrics per transaction size
    """
    # Initialize storage for I/O operations by size
    io_ops_by_size = {
        'read': defaultdict(list),
        'write': defaultdict(list)
    }
    
    with otf2.reader.open(trace_file_path) as trace:
        # Find MPI I/O paradigm definition
        mpi_io_paradigm = None
        for trace_paradigm in trace.definitions.io_paradigms:
            if trace_paradigm.identification == paradigm.value:
                mpi_io_paradigm = trace_paradigm
                break
                
        if mpi_io_paradigm is None:
            raise ValueError("Given MPI I/O paradigm not found in trace")
            
        # Track ongoing operations by matching_id
        ongoing_ops = {}
        
        for location, event in trace.events:
            if isinstance(event, otf2.events.IoOperationBegin):
                # Only track MPI I/O operations
                if event.handle.io_paradigm == mpi_io_paradigm:
                    ongoing_ops[event.matching_id] = {
                        'start_time': event.time,
                        'bytes_requested': event.bytes_request,
                        'mode': event.mode
                    }
                    
            elif isinstance(event, otf2.events.IoOperationComplete):
                if event.matching_id in ongoing_ops:
                    start_info = ongoing_ops[event.matching_id]
                    duration = event.time - start_info['start_time']
                    bytes_transferred = event.bytes_result
                    
                    # Calculate bandwidth in MB/s
                    if duration > 0:
                        bandwidth = (bytes_transferred / (1024 * 1024)) / (duration * 1e-9)
                        
                        # Store operation details grouped by size
                        if start_info['mode'] == otf2.IoOperationMode.READ:
                            op_type = 'read' 
                        elif start_info['mode'] == otf2.IoOperationMode.WRITE:
                            op_type = 'write'
                        else:
                            print(f"Unknown operation mode: {start_info['mode']}")
                            continue
                    
                        io_ops_by_size[op_type][bytes_transferred].append({
                            'bytes': bytes_transferred,
                            'duration_ns': duration,
                            'bandwidth_mbs': bandwidth
                        })
                    
                    del ongoing_ops[event.matching_id]

    # Calculate metrics per size
    metrics = {
        'read': {},
        'write': {}
    }
    
    for op_type in ['read', 'write']:
        for size, operations in io_ops_by_size[op_type].items():
            metrics[op_type][size] = {
                'count': len(operations),
                'total_bytes': sum(op['bytes'] for op in operations),
                'total_duration_ns': sum(op['duration_ns'] for op in operations),
                'avg_bandwidth_mbs': 0,
                'min_bandwidth_mbs': min((op['bandwidth_mbs'] for op in operations), default=0),
                'max_bandwidth_mbs': max((op['bandwidth_mbs'] for op in operations), default=0),
                'std_bandwidth_mbs': 0
            }
            
            # Calculate average bandwidth
            if metrics[op_type][size]['total_duration_ns'] > 0:
                metrics[op_type][size]['avg_bandwidth_mbs'] = \
                    (metrics[op_type][size]['total_bytes'] / (1024 * 1024)) / \
                    (metrics[op_type][size]['total_duration_ns'] * 1e-9)
            
            # Calculate standard deviation of bandwidth
            if len(operations) > 1:
                mean = metrics[op_type][size]['avg_bandwidth_mbs']
                variance = sum((op['bandwidth_mbs'] - mean) ** 2 for op in operations) / (len(operations) - 1)
                metrics[op_type][size]['std_bandwidth_mbs'] = variance ** 0.5
    
    return metrics

In [34]:
calculate_mpi_io_bandwidth_by_size(otf2_trace, Otf2Paradigm.MPIIO)

{'read': {42515280: {'count': 40,
   'total_bytes': 1700611200,
   'total_duration_ns': 6687167402,
   'avg_bandwidth_mbs': 242.5285814061953,
   'min_bandwidth_mbs': 75.58547166298375,
   'max_bandwidth_mbs': 298.51505473358645,
   'std_bandwidth_mbs': 34.97532679883168}},
 'write': {42515280: {'count': 40,
   'total_bytes': 1700611200,
   'total_duration_ns': 10879508135,
   'avg_bandwidth_mbs': 149.07192526611522,
   'min_bandwidth_mbs': 64.62763295368534,
   'max_bandwidth_mbs': 162.2347998607631,
   'std_bandwidth_mbs': 15.680638135255899}}}